# Thermodynamics of Urea's Effects on the Solubility of Polyethylene Glycol
Stefan Hervø-Hansen<sup>a</sup>, Kazuya Okita<sup>a</sup>, Kento Kasahara<sup>a</sup>, and Nobuyuki Matubayasi<sup>a,</sup><br><br>
<sup>a</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonaka, Osaka 560-8531, Japan.<br>
<sup></sup> Correspondence may be addressed to: stefan@cheng.es.osaka-u.ac.jp and nobuyuki@cheng.es.osaka-u.ac.jp.

## Part 1: Simulations

### Introduction
Here we aim to provide a detailed thermodynamic analysis of how urea influence the solvation of polyethylene glycol (PEG). By utilizing molecular dynamics simulations, we can gain atomic insight into the mechanism which underpins the change in excess chemical potential of PEG with the addition of urea. Understanding these effects is crucial for applications in biochemistry and materials science, where PEG and its derivatives are widely used. The following sections detail the methods and materials employed in our simulations and analyses.

### Methods & Materials
Molecular dynamics simulations were conducted using the OpenMM (8.0)[<sup>1</sup>](#fn1) software package. The details of these simulations can be found in the [Part 1 Jupyter notebook](Simulations.ipynb). For the simulation of PEG, a CHARMM-derived force field (C35r) was utilized, which has previously been shown to reproduce the hydrodynamic radii and shape anisotropy of PEG[<sup>2</sup>](#fn2). The PEG force field was combined with the SPC/E force field for water[<sup>3</sup>](#fn3) and a Kirkwood-Buff derived force field for urea[<sup>4</sup>](#fn4).

The isothermal-isobaric ensemble was sampled using a combination of a "Middle" discretization Langevin leap-frog integrator[<sup>5,</sup>](#fn5)[<sup>6</sup>](#fn6) and a Monte Carlo barostat[<sup>7,</sup>](#fn7)[<sup>8</sup>](#fn8). The trajectories were analyzed using MDTraj[<sup>9</sup>](#fn9) for structural properties, while ERmod[<sup>10</sup>](#fn10) was used for the calculation of solvation free energies. The calculation of solvation free energy can be found in the [Part 2 Jupyter notebook](ERmod.ipynb) and the analysis of data can be found in [Part 3 Jupyter notebook](Analysis.ipynb)

### References
1. <span id="fn1"> P. Eastman, et al., OpenMM 8: Molecular Dynamics Simulation with Machine Learning Potentials. J. Phys. Chem. B 128, 109–116 (2023). </span><br>
2. <span id="fn2"> H. Lee, R. M. Venable, A. D. MacKerell Jr., R. W. Pastor, Molecular Dynamics Studies of Polyethylene Oxide and Polyethylene Glycol: Hydrodynamic Radius and Shape Anisotropy. Biophysical Journal 95, 1590–1599 (2008). </span><br>
3. <span id="fn3"> H. J. C. Berendsen, J. R. Grigera, T. P. Straatsma, The missing term in effective pair potentials. J. Phys. Chem. 91, 6269–6271 (1987). </span><br>
4. <span id="fn4"> S. Weerasinghe, P. E. Smith, A Kirkwood−Buff Derived Force Field for Mixtures of Urea and Water. J. Phys. Chem. B 107, 3891–3898 (2003). </span><br>
5. <span id="fn5"> B. Leimkuhler, C. Matthews, Efficient molecular dynamics using geodesic integration and solvent–solute splitting. Proc. R. Soc. A. 472, 20160138 (2016). </span><br>
6. <span id="fn6"> Z. Zhang, X. Liu, K. Yan, M. E. Tuckerman, J. Liu, Unified Efficient Thermostat Scheme for the Canonical Ensemble with Holonomic or Isokinetic Constraints via Molecular Dynamics. J. Phys. Chem. A 123, 6056–6079 (2019). </span><br>
7. <span id="fn7"> K.-H. Chow, D. M. Ferguson, Isothermal-isobaric molecular dynamics simulations with Monte Carlo volume sampling. Computer Physics Communications 91, 283–289 (1995). </span><br>
8. <span id="fn8"> J. Åqvist, P. Wennerström, M. Nervall, S. Bjelic, B. O. Brandsdal, Molecular dynamics simulations of water and biomolecules with a Monte Carlo constant pressure algorithm. Chemical Physics Letters 384, 288–294 (2004). </span><br>
9. <span id="fn9"> R. T. McGibbon, et al., MDTraj: A Modern Open Library for the Analysis of Molecular Dynamics Trajectories. Biophysical Journal 109, 1528–1532 (2015). </span><br>
10. <span id="fn10"> S. Sakuraba, N. Matubayasi, Ermod: Fast and versatile computation software for solvation free energy with approximate theory of solutions. J. Comput. Chem. 35, 1592–1608 (2014). </span><br>

## Import of Python Modules & Auxiliary Functions

In [ ]:
# Notebook dependent libs
import numpy as np
import os
from distutils.spawn import find_executable
import parmed as pmd
from openmm import app
import mdtraj as md

# Simulation specific libs
# from openmm import app
# import openmm as mm
# import mdtraj as md

# Check for external programs
if None in [find_executable('packmol')]:
    print('WARNING: External program missing!')

homedir = !pwd
homedir = homedir[0]
workdir = '/home/users/eau/PEO_TMP' # RCCS
print(homedir)

## Molecular Dynamics Simulations
To provide a rigorous thermodynamic description of the effects of urea on PEG, as described in the paper, we adopt an unconventional simulation strategy to calculate the difference in chemical potential of PEG upon the addition of urea. In specific, the simulation strategy contains three steps:
1. Generation of a PEG conformational ensemble in a given solvent condition.
2. Generation of solvent configurations in the absence ($\lambda=0$) of PEG.
3. Generation of solvent configurations in the presence ($\lambda=1$) of a fixed PEG configuration.

The steps of the simulation strategy are outlined throughout the Jupyter Notebook.

### Simulation settings
The number of steps and the output frequency of the simulations are controlled via the `states` variable. The dictionary contains 3 entries, namely `conf`, `sol`, and `ref` which are referring to the simulations for generating the PEG conformational ensemble in water, the simulation of the solvated state ($\lambda=1$) with frozen PEG, and the reference state ($\lambda=0$) in which PEG is absent, respectively. 

The number of conformations chosen via `NConfs` is picked via linear spaced sampling from the conformational ensemble generated.


### OpenMM settings & setup
The notebook is designed to be fully automated in the construction of simulation input files for OpenMM using the python API. If you wish to edit the default OpenMM script one simply needs to edit the variable `openmm_script`. For example, one can edit the integration scheme and its parameters set in the variable `integrator` as well as edit the barostat as currently determined from `mm.MonteCarloBarostat`. Additionally, one may change the non-bonded methods and their cutoffs in the `system` variable with the option of adding Lennard-Jones switching functions via the `forces` variable. Finally, one may edit the number of minimization (`sim.minimizeEnergy`) and equilibration steps conducted as well as choose whether the simulation should be conducted on GPUs or CPUs via the `platform` and `properties` variables.

### Submit script
Due to the larger amount of simulations, the notebook can write submission script for servers employing job scheduling. The notebook was origionally run on a linux machine utilizing PBS (for a quick guide see [here](https://latisresearch.umn.edu/creating-a-PBS-script)). However, the bash script containing the submission code may be edited to utilize Slurm instead (documentation [here](https://slurm.schedmd.com)) by changing the variable `submit_script` and by executing the commands `!sbatch submit.pbs`, `!sbatch submit_sol.pbs`, and `!sbatch submit_ref.pbs` instead of `qsub`.

In [ ]:
states = { # State of simulations, (outFreq is steps per frame)
          'conf':{'Nsteps': 500000000, 'OutFreq': 1000}, # 1000 nanoseconds, 500.000 frames
          'sol': {'Nsteps':  25000000, 'OutFreq':  500}, #   50 nanoseconds,  50.000 frames
          'ref': {'Nsteps':  50000000, 'OutFreq':  500}, #  100 nanoseconds, 100.000 frames
         }

nmers = [36] # PEG polymer length
Nparticles = {       # Number of PEG and water molecules. Salt is calculated based on concentration input
    'PEG': 1,
    'Water': 10000,
}
NConfs = 100

urea_reference_concentrations = { 
   0.00: {'P0':    0, 'P1': 378, 'P2': 1472},
   2.00: {'P0':  378, 'P1':   0, 'P2': 1472},
   6.00: {'P0': 1472, 'P1':   0, 'P2':  378},
}

GENERATE_SOLUTE     = False   # Generate the structual ensemble of PEG
GENERATE_REFERENCES = False   # Generate the reference conditions
GENERATE_SOLUTIONS  = False   # Generate the solution conditions

### Step 1. Generation of Solute Configurations

#### Construction structure files

In [ ]:
%cd -q $homedir

packmol_script="""
tolerance 2.0
filetype pdb
output PEG_{nmer}_urea_{conc}.pdb

add_box_sides 1.0

structure {homedir}/PDB_files/PEO-{nmer}-mer.pdb
        number {N_PEG}
        fixed 35. 35. 35. 0. 0. 0.
        centerofmass
end structure

{urea}structure {homedir}/PDB_files/urea.pdb
{urea}        number {N_urea}
{urea}        inside cube 0. 0. 0. 70.
{urea}end structure
"""

if GENERATE_SOLUTE:
    for nmer in nmers:
        nmerdir = 'PEG{}mer'.format(nmer)
        for urea_reference in urea_reference_concentrations.items():
            C = '{0:.2f}'.format(urea_reference[0])
            %cd -q $homedir/Simulations/$nmerdir/$C/Solute
    
            # Packmol Input
            with open('packmol.in', 'w') as f:
                # Fix for no urea
                if float(C) == 0:
                    ureaFix='#'
                    Nurea = 0
                else:
                    ureaFix=''
                    Nurea = urea_reference[1]['P0']
                f.write(packmol_script.format(N_PEG=Nparticles['PEG'], nmer=nmer, urea=ureaFix,
                                              homedir=homedir, N_urea=Nurea, conc=C))
            f.close()
            !/home/z44785r/packmol-20.14.0/packmol < packmol.in
            
            print('Wrote initial configurations and topology files to'+os.getcwd())

#### Molecular dynamics setup using OpenMM

In [ ]:
%cd -q $homedir

openmm_script="""# Imports
import sys
import os
import openmm as mm
from openmm import app
from openmm import unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
pdb = app.PDBFile('PEG_{nmer}_urea_{conc}.pdb')
forcefield = app.ForceField('{workdir}/Force_fields/peg.xml',
                            '{workdir}/Force_fields/spce.xml',
                            '{workdir}/Force_fields/urea.xml',
                         
modeller = app.Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens(forcefield, pH=7.0)
modeller.addSolvent(forcefield, model='spce', numAdded=10000, neutralize=False)

app.PDBFile.writeFile(modeller.topology, modeller.positions, open('PEG_{nmer}_urea_{conc}_hydrated.pdb', 'w'))

# Creating system
print('Creating OpenMM System')
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, ewaldErrorTolerance=0.0005, switchDistance=1*u.nanometer,
                                 nonbondedCutoff=1.2*u.nanometers, constraints=app.HBonds, rigidWater=True)

# Calculating total mass of system
total_mass = u.sum([system.getParticleMass(i) for i in range(system.getNumParticles())])
        
# Temperature-coupling by leap frog (BAOAB) Langevin integrator (NVT)
integrator = mm.LangevinMiddleIntegrator(298.15*u.kelvin, 1.0/u.picoseconds, 2.0*u.femtoseconds)

# Pressure-coupling by a Monte Carlo Barostat (NPT)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 298.15*u.kelvin, 25))

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0'}}

# Create the Simulation object
sim = app.Simulation(modeller.topology, system, integrator, platform, properties)

# Set the particle positions
sim.context.setPositions(modeller.positions)

# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(tolerance=1*u.kilojoule/u.mole, maxIterations=1000000)
    
# Draw initial MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(50000000)  # 50000000*2 fs = 100 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output.dat', {outFreq}, totalSteps={Nsteps}+50000000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    systemMass=total_mass, remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})
"""           

N_simulations = 0
if GENERATE_SOLUTE:
    for nmer in nmers:
        nmerdir = 'PEG{}mer'.format(nmer)
        for urea_reference in urea_reference_concentrations.items():
            C = '{0:.2f}'.format(urea_reference[0])
            %cd -q $homedir/Simulations/$nmerdir/$C/Solute
                
            with open('openMM.py', 'w') as f:
                f.write(openmm_script.format(workdir=workdir, Nsteps=states['conf']['Nsteps'],
                                             nmer=nmer, outFreq=states['conf']['OutFreq'],
                                             conc=C))
            f.close()
            print('Wrote run_openMM.py files to '+os.getcwd())
            N_simulations+=1

print('Simulations about to be submitted: {}'.format(N_simulations))

#### Submit script

In [ ]:
%cd -q $homedir

submit_script="""#!/bin/bash
## RCCS
#PBS -l select=1:ncpus=1:mpiprocs=1:ompthreads=1:jobtype=gpu:ngpus=1
#PBS -l walltime=120:00:00
#PBS -N Flexable
#PBS -e run.err
#PBS -o run.out

source ~/.bashrc
source ~/.bash_profile

# export CUDA_VISIBLE_DEVICES=0

cd {path}

python openMM.py
"""

master_submit_script="""#!/bin/bash
read -p "Will submit {N_simulations} simulations. Do you want to proceed? (yes/no) " yn

case $yn in 
    yes ) echo submitting...;;
    no ) echo exiting...;
         exit;;
    * ) echo invalid response;
        exit 1;;
esac

"""
if GENERATE_SOLUTE:
    with open('Simulations/master_solute.sh', 'w') as ff:
        ff.write(master_submit_script.format(N_simulations=N_simulations))
    
        for nmer in nmers:
            nmerdir = 'PEG{}mer'.format(nmer)
            for urea_reference in urea_reference_concentrations.items():
                C = '{0:.2f}'.format(urea_reference[0])
                %cd -q $homedir/Simulations/$nmerdir/$C/Solute
                        
                simpath = os.getcwd()
                simpath = simpath.split(sep='PEO_TMP')[1]
                simpath = workdir+simpath
                with open('submit.pbs', 'w') as f:
                    f.write(submit_script.format(path=simpath))
                f.close()
                s =  'cd {}\njsub submit.pbs\n\n'.format(simpath) # RCCS
                ff.write(s)
    ff.close()

#### Post simulation trajectory processing

In [ ]:
%cd -q $homedir

if GENERATE_SOLUTE:
    for nmer in nmers:
        nmerdir = 'PEG{}mer'.format(nmer)
        for urea_reference in urea_reference_concentrations.items():
            C = '{0:.2f}'.format(urea_reference[0])
            %cd -q $homedir/Simulations/$nmerdir/$C/Solute
            
            pdb = md.load_pdb('PEG_{nmer}_urea_{conc}_hydrated.pdb'.format(nmer=nmer, conc=C))
            PEG_indices = pdb.topology.select("resname =~ 'PG*'")
            traj = md.load_xtc('trajectory.xtc',
                               top='PEG_{nmer}_urea_{conc}_hydrated.pdb'.format(nmer=nmer, conc=C),
                               atom_indices=PEG_indices)
            traj.save_xtc('trajectory_dry.xtc', force_overwrite=True)

### Step 2. Generation of Solvent Configurations - Reference System (λ = 0)
#### Generation of initial configuration using Packmol

In [ ]:
%cd -q $homedir

packmol_script="""
tolerance 2.0
filetype pdb
output urea_{conc}.pdb

add_box_sides 1.0

structure {homedir}/PDB_files/urea.pdb
        number {N_urea}
        inside cube 0. 0. 0. 70.
end structure
"""

if GENERATE_REFERENCES:
    for urea_reference in urea_reference_concentrations.items():
        C = '{0:.2f}'.format(urea_reference[0])
        %cd -q $homedir/Simulations/References/$C
        
        if urea_reference[0] > 0:
            with open('packmol.in', 'w') as f:
                Nurea = urea_reference[1]['P0']
                f.write(packmol_script.format(homedir=homedir, N_urea=Nurea, conc=C))
            f.close()
            !/home/z44785r/packmol-20.14.0/packmol < packmol.in
            pdb = app.PDBFile('urea_{conc}.pdb'.format(conc=C))
            modeller = app.Modeller(pdb.topology, pdb.positions)
        
        else: # Fix for pure water
            modeller = app.Modeller(app.Topology(), [])
                
        forcefield = app.ForceField(homedir+'/Force_fields/peg.xml', homedir+'/Force_fields/spce.xml',
                                    homedir+'/Force_fields/urea.xml')
        modeller.addSolvent(forcefield, model='spce', numAdded=Nparticles['Water'], neutralize=False)
        app.PDBFile.writeFile(modeller.topology, modeller.positions,
                              file=open('urea_{conc}.pdb'.format(conc=C), 'w'))
        
        print('Wrote initial configurations to'+os.getcwd())

#### Molecular dynamics setup using OpenMM

In [ ]:
%cd -q $homedir

openmm_script="""# Imports
import sys
import os
import openmm as mm
from openmm import app
from openmm import unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
pdb = app.PDBFile('urea_{conc}.pdb')
forcefield = app.ForceField('{workdir}/Force_fields/peg.xml',
                            '{workdir}/Force_fields/spce.xml',
                            '{workdir}/Force_fields/urea.xml',

# Creating system
print('Creating OpenMM System')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, ewaldErrorTolerance=0.0005, switchDistance=1*u.nanometer,
                                 nonbondedCutoff=1.2*u.nanometers, constraints=app.HBonds, rigidWater=True)

# Calculating total mass of system
total_mass = u.sum([system.getParticleMass(i) for i in range(system.getNumParticles())])
        
# Temperature-coupling by leap frog (BAOAB) Langevin integrator (NVT)
integrator = mm.LangevinMiddleIntegrator(298.15*u.kelvin, 1.0/u.picoseconds, 2.0*u.femtoseconds)

# Pressure-coupling by a Monte Carlo Barostat (NPT)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 298.15*u.kelvin, 25))

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0'}}

# Create the Simulation object
sim = app.Simulation(pdb.topology, system, integrator, platform, properties)

# Set the particle positions
sim.context.setPositions(pdb.positions)

# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(tolerance=1*u.kilojoule/u.mole, maxIterations=1000000)
    
# Draw initial MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(5000000)  # 5000000*2 fs = 10.0 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output.dat', {outFreq}, totalSteps={Nsteps}+5000000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    systemMass=total_mass, remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})
"""

N_simulations = 0
if GENERATE_REFERENCES:
    for urea_reference in urea_reference_concentrations.items():
        C = '{0:.2f}'.format(urea_reference[0])
        %cd -q $homedir/Simulations/References/$C
                
        with open('openMM.py', 'w') as f:
            f.write(openmm_script.format(Nsteps=states['ref']['Nsteps'], workdir=workdir, conc=C,
                                         outFreq=states['ref']['OutFreq']))
        f.close()
        N_simulations+=1
        print('Wrote run_openMM.py files to '+os.getcwd())

print('Simulations about to be submitted: {}'.format(N_simulations))

#### Submit script

In [ ]:
%cd -q $homedir

submit_script="""#!/bin/bash
## RCCS
#PBS -l select=1:ncpus=1:mpiprocs=1:ompthreads=1:jobtype=gpu:ngpus=1
#PBS -l walltime=24:00:00
#PBS -N Reference
#PBS -e run.err
#PBS -o run.out

source ~/.bashrc
source ~/.bash_profile

# export CUDA_VISIBLE_DEVICES=0

cd {path}

python openMM.py
"""

master_submit_script="""#!/bin/bash
read -p "Will submit {N_simulations} simulations. Do you want to proceed? (yes/no) " yn

case $yn in 
    yes ) echo submitting...;;
    no ) echo exiting...;
         exit;;
    * ) echo invalid response;
        exit 1;;
esac

"""

if GENERATE_REFERENCES:
    with open('Simulations/master_reference.sh', 'w') as ff:
        ff.write(master_submit_script.format(N_simulations=N_simulations))
        
        for urea_reference in urea_reference_concentrations.items():
            C = '{0:.2f}'.format(urea_reference[0])
            %cd -q $homedir/Simulations/References/$C
                
            simpath = os.getcwd()
            simpath = simpath.split(sep='PEO_TMP')[1]
            simpath = workdir+simpath
            with open('submit.pbs', 'w') as f:
                f.write(submit_script.format(path=simpath))
            f.close()
            s =  'cd {}\njsub submit.pbs\n\n'.format(simpath) # RCCS
            ff.write(s)
                
        ff.close()

### Step 3. Generation of Solvent Configurations at Constant Solute Configuration - Solution System (λ = 1)
#### Construction of topology and structure files
Fully automated construction of topologies files in Amber format and initial configurations using packmol. No major important parameters to edit in the following code.

In [ ]:
%cd -q $homedir

packmol_script="""
tolerance 2.0
filetype pdb
output PEG_{nmer}_urea.pdb
add_amber_ter

add_box_sides 0.1

structure conf.pdb
        number 1
        fixed 35. 35. 35. 0. 0. 0.
        centerofmass
end structure

structure {homedir}/PDB_files/urea.pdb
        number {N_urea}
        inside cube 0. 0. 0. 70.
end structure
"""

if GENERATE_SOLUTIONS:
    for nmer in nmers:
        nmerdir = 'PEG{}mer'.format(nmer)
        for urea_reference, perturb in urea_reference_concentrations.items():
            C = '{0:.2f}'.format(urea_reference)
            %cd -q $homedir/Simulations/$nmerdir/$C
                
            peg_traj = md.load_xtc('Solute/trajectory_dry.xtc',
                                   top=homedir+'/PDB_files/PEO-{nmer}-mer.pdb'.format(nmer=nmer))
            peg_traj = peg_traj[peg_traj.n_frames//2::peg_traj.n_frames//(2*NConfs)]
            assert len(peg_traj) == NConfs, 'The number of frames loaded is inconsistant with the number of desired conformations'
            
            for P in ['P0', 'P1', 'P2']:
                for conf in range(NConfs):
                    %cd -q $homedir/Simulations/$nmerdir/$C/$P/$conf
                    mol = peg_traj[conf]
                    mol.save('conf.pdb')
                    if perturb[P] == 0:
                        pdb = app.PDBFile('conf.pdb'.format(nmer=nmer, conf=conf))
                        modeller = app.Modeller(pdb.topology, pdb.positions)
                        forcefield = app.ForceField(homedir+'/Force_fields/peg.xml',
                                                    homedir+'/Force_fields/spce.xml',
                                                    homedir+'/Force_fields/urea.xml')
                        modeller.addSolvent(forcefield, model='spce', numAdded=Nparticles['Water'],
                                            neutralize=False)
                        app.PDBFile.writeFile(modeller.topology, modeller.positions,
                                              file=open('PEG_{nmer}_urea.pdb'.format(nmer=nmer), 'w'))
                    else:
                        with open('packmol.in', 'w') as f:
                                f.write(packmol_script.format(homedir=homedir, N_urea=perturb[P], nmer=nmer))
                        f.close()
                        !/home/z44785r/packmol-20.14.0/packmol < packmol.in
                        pdb = app.PDBFile('PEG_{nmer}_urea.pdb'.format(nmer=nmer))
                        modeller = app.Modeller(pdb.topology, pdb.positions)
                        forcefield = app.ForceField(homedir+'/Force_fields/peg.xml',
                                                    homedir+'/Force_fields/spce.xml',
                                                    homedir+'/Force_fields/urea.xml')
                        modeller.addSolvent(forcefield, model='spce', numAdded=Nparticles['Water'],
                                            neutralize=False)
                        app.PDBFile.writeFile(modeller.topology, modeller.positions,
                                              file=open('PEG_{nmer}_urea.pdb'.format(nmer=nmer), 'w'))
                    
                # Collect it all for λ=1
                %cd -q $homedir/Simulations/$nmerdir/$C
                system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, ewaldErrorTolerance=0.0005,
                                                     nonbondedCutoff=1.2*pmd.unit.nanometers, rigidWater=False)
                mol = pmd.openmm.load_topology(modeller.topology, system, modeller.positions)
                mol.save('{P}/PEG_{nmer}_urea.parm7'.format(P=P, nmer=nmer), overwrite=True)
                print('Wrote initial configurations and topology files to'+os.getcwd()+'/'+P)

#### Construction of molecular dynamics input for OpenMM

In [ ]:
%cd -q $homedir
openmm_script="""# Imports
import sys
import os
import openmm as mm
from openmm import app
from openmm import unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
pdb = app.PDBFile('PEG_{nmer}_urea.pdb')
forcefield = app.ForceField('{workdir}/Force_fields/peg.xml',
                            '{workdir}/Force_fields/spce.xml',
                            '{workdir}/Force_fields/urea.xml')
    
# Find all PEG atoms   
PEG_atoms = []
for residue in pdb.topology.residues():
    if residue.name in ['PGH', 'PGM', 'PGT']:
        for atom in residue.atoms():
            PEG_atoms.append(atom)

# Creating system
print('Creating OpenMM System')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, ewaldErrorTolerance=0.0005, switchDistance=1*u.nanometer,
                                 nonbondedCutoff=1.2*u.nanometers, constraints=None, rigidWater=True)

# Calculating total mass of system
total_mass = u.sum([system.getParticleMass(i) for i in range(system.getNumParticles())])

# Freeze PEG atoms by setting mass to 0
for atom in PEG_atoms:
    system.setParticleMass(atom.index, 0.000*u.dalton)
        
# Temperature-coupling by leap frog "middle"-discretization Langevin integrator (NVT)
integrator = mm.LangevinMiddleIntegrator(298.15*u.kelvin, 1.0/u.picoseconds, 2.0*u.femtoseconds)

# Pressure-coupling by a Monte Carlo Barostat (NPT)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 298.15*u.kelvin, 25))

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0'}}

# Create the Simulation object
sim = app.Simulation(pdb.topology, system, integrator, platform, properties)

# Set the particle positions
sim.context.setPositions(pdb.positions)

# Minimize the energy
print('Minimizing energy...')
sim.minimizeEnergy()

# Save minimized coordinates
positions = sim.context.getState(getPositions=True).getPositions()

# Reinitialize integrator, system, and simulation with correct constraints
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, ewaldErrorTolerance=0.0005, switchDistance=1*u.nanometer,
                                 nonbondedCutoff=1.2*u.nanometers, constraints=app.HBonds, rigidWater=True)
for atom in PEG_atoms:
    system.setParticleMass(atom.index, 0.000*u.dalton)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 298.15*u.kelvin, 25))
integrator = mm.LangevinMiddleIntegrator(298.15*u.kelvin, 1.0/u.picoseconds, 2.0*u.femtoseconds)
sim = app.Simulation(pdb.topology, system, integrator, platform, properties)
sim.context.setPositions(positions)
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(5000000)  # 5000000*2 fs = 10.0 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output.dat', {outFreq}, totalSteps={Nsteps}+5000000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    systemMass=total_mass, remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})

# Create a dictionary to store all forces
forces = {{sim.system.getForce(index).__class__.__name__: sim.system.getForce(index) for index in range(sim.system.getNumForces())}}

# Print PME information
print('''
PARTICLE MESH EWALD PARAMETERS
Separation parameter: {{}}
Number of grid points along the X axis: {{}}
Number of grid points along the Y axis: {{}}
Number of grid points along the Z axis: {{}}
'''.format(*forces['NonbondedForce'].getPMEParametersInContext(sim.context)))
"""

if GENERATE_SOLUTIONS:
    N_simulations = 0
    for nmer in nmers:
        nmerdir = 'PEG{}mer'.format(nmer)
        for urea_reference, perturb in urea_reference_concentrations.items():
            C = '{0:.2f}'.format(urea_reference)
            for P in ['P0', 'P1', 'P2']:
                for conf in range(NConfs):
                    %cd -q $homedir/Simulations/$nmerdir/$C/$P/$conf
                    
                    with open('openMM.py', 'w') as f:
                        f.write(openmm_script.format(Nsteps=states['sol']['Nsteps'], workdir=workdir,
                                                     outFreq=states['sol']['OutFreq'], nmer=nmer))
                    f.close()
                    N_simulations+=1
                    print('Wrote run_openMM.py files to '+os.getcwd())
    
    print('Simulations about to be submitted: {}'.format(N_simulations))

#### Submit script

In [ ]:
%cd -q $homedir

submit_script="""#!/bin/bash
## RCCS
#PBS -l select=1:ncpus=1:mpiprocs=1:ompthreads=1:jobtype=gpu:ngpus=1
#PBS -l walltime=24:00:00
#PBS -N Solution
#PBS -e run.err
#PBS -o run.out

source ~/.bashrc
source ~/.bash_profile

# export CUDA_VISIBLE_DEVICES=0

cd {path}

python openMM.py
"""

master_submit_script="""#!/bin/bash
read -p "Will submit {N_simulations} simulations. Do you want to proceed? (yes/no) " yn

case $yn in 
    yes ) echo submitting...;;
    no ) echo exiting...;
         exit;;
    * ) echo invalid response;
        exit 1;;
esac

"""

if GENERATE_SOLUTIONS:
    with open('Simulations/master_solution.sh', 'w') as ff:
        ff.write(master_submit_script.format(N_simulations=N_simulations))
        
        for nmer in nmers:
            nmerdir = 'PEG{}mer'.format(nmer)
            for urea_reference, perturb in urea_reference_concentrations.items():
                C = '{0:.2f}'.format(urea_reference)
                for P in ['P0', 'P1', 'P2']:
                    for conf in range(NConfs):
                        %cd -q $homedir/Simulations/$nmerdir/$C/$P/$conf
                    
                        simpath = os.getcwd()
                        simpath = simpath.split(sep='PEO_TMP')[1]
                        simpath = workdir+simpath
                        with open('submit.pbs', 'w') as f:
                            f.write(submit_script.format(path=simpath))
                        f.close()
                        s =  'cd {}\njsub submit.pbs\n\n'.format(simpath) # RCCS
                        ff.write(s)
    ff.close()